In [ ]:
from ppdai_utils import *
import pickle

cdict, emat = embedding('/files/faust/COMPETITION/ppdai/char_embed.txt')
questions = pickle.load(open('/files/faust/COMPETITION/ppdai/questions.pkl', 'rb'))
trainpair = readtrain_extend()

In [ ]:
charlen = [len(wc['cchars']) for _,wc in questions.items()]
print(max(charlen))
charlen = [len(wc['cwords']) for _,wc in questions.items()]
print(max(charlen))

In [4]:
MAX_SEQUENCE_LENGTH = 60
trainpair_idx = []
for q0, q1, l in trainpair:
    q0_chars = questions[q0]['cchars']
    q1_chars = questions[q1]['cchars']
    q0_idx = sent2idx(q0_chars, cdict, MAX_SEQUENCE_LENGTH)
    q1_idx = sent2idx(q1_chars, cdict, MAX_SEQUENCE_LENGTH)
    trainpair_idx.append((q0_idx, q1_idx, l))
print(len(trainpair_idx))

508772


In [5]:
EMBEDDING_DIM = 300
DROPOUT = 0.5

In [11]:
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv1D, MaxPooling1D
from keras.layers import Dropout, BatchNormalization
from keras.layers import concatenate, Flatten
from keras.layers import Lambda
embedding_layer = Embedding(len(cdict),
                            EMBEDDING_DIM,
                            weights=[emat],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

layer_conv_1_1 = Conv1D(64, 1, activation='tanh')
layer_conv_1_2 = Conv1D(64, 2, activation='tanh')
layer_conv_1_3 = Conv1D(64, 3, activation='tanh')
layer_conv_1_5 = Conv1D(64, 5, activation='tanh')
layer_fc_1 = Dense(256, activation="tanh")
layer_lr = Dense(1, activation='sigmoid')

input_q1 = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
input_q2 = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
q1_embed = embedding_layer(input_q1)
q2_embed = embedding_layer(input_q2)

# for q1
q1_conv_1_1 = layer_conv_1_1(q1_embed)
q1_conv_1_2 = layer_conv_1_2(q1_embed)
q1_conv_1_3 = layer_conv_1_3(q1_embed)
q1_conv_1_5 = layer_conv_1_5(q1_embed)
q1_pool_1_1 = MaxPooling1D(3)(q1_conv_1_1)
q1_pool_1_1 = Flatten()(q1_pool_1_1)
q1_pool_1_2 = MaxPooling1D(3)(q1_conv_1_2)
q1_pool_1_2 = Flatten()(q1_pool_1_2)
q1_pool_1_3 = MaxPooling1D(3)(q1_conv_1_3)
q1_pool_1_3 = Flatten()(q1_pool_1_3)
q1_pool_1_5 = MaxPooling1D(3)(q1_conv_1_5)
q1_pool_1_5 = Flatten()(q1_pool_1_5)
q1_merged = concatenate([q1_pool_1_1, q1_pool_1_2, q1_pool_1_3, q1_pool_1_5])
q1_fc_1 = layer_fc_1(q1_merged)

# for q2
q2_conv_1_1 = layer_conv_1_1(q2_embed)
q2_conv_1_2 = layer_conv_1_2(q2_embed)
q2_conv_1_3 = layer_conv_1_3(q2_embed)
q2_conv_1_5 = layer_conv_1_5(q2_embed)
q2_pool_1_1 = MaxPooling1D(3)(q2_conv_1_1)
q2_pool_1_1 = Flatten()(q2_pool_1_1)
q2_pool_1_2 = MaxPooling1D(3)(q2_conv_1_2)
q2_pool_1_2 = Flatten()(q2_pool_1_2)
q2_pool_1_3 = MaxPooling1D(3)(q2_conv_1_3)
q2_pool_1_3 = Flatten()(q2_pool_1_3)
q2_pool_1_5 = MaxPooling1D(3)(q2_conv_1_5)
q2_pool_1_5 = Flatten()(q2_pool_1_5)
q2_merged = concatenate([q2_pool_1_1, q2_pool_1_2, q2_pool_1_3, q2_pool_1_5])
q2_fc_1 = layer_fc_1(q2_merged)

# fusing q0, q1
q12 = Lambda(lambda x: x ** 2)(q1_fc_1 - q2_fc_1)
pred = layer_lr(q12)

model = Model([input_q1, input_q2], pred)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_crossentropy'])

AttributeError: 'Tensor' object has no attribute '_keras_history'

In [ ]:
EMBEDDING_DIM = 300
DROPOUT = 0.5
MAX_SEQUENCE_LENGTH = 60
question1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
question2 = Input(shape=(MAX_SEQUENCE_LENGTH,))

q1 = Embedding(len(cdict),
                 EMBEDDING_DIM,
                 weights=[word_embedding_matrix],
                 input_length=MAX_SEQUENCE_LENGTH,
                 trainable=False)(question1)
q1 = Conv1D(256, 5, activation='relu')(q1)
q1 = MaxPooling1D(5)(q1)
q1 = Flatten()(q1)
q1 = Dense(128, activation="relu")(q1)
#q1 = Dropout(0.5)(q1)

q2 = Embedding(len(cdict),
                 EMBEDDING_DIM,
                 weights=[word_embedding_matrix],
                 input_length=MAX_SEQUENCE_LENGTH,
                 trainable=False)(question2)
q2 = Conv1D(256, 5, activation='relu')(q2)
q2 = MaxPooling1D(5)(q2)
q2 = Flatten()(q2)
q2 = Dense(128, activation="relu")(q2)
#q2 = Dropout(0.5)(q2)
merged = concatenate([q1,q2])
is_duplicate = Dense(1, activation='sigmoid')(merged)
model = Model(inputs=[question1,question2], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_crossentropy'])

In [ ]:
# train.test split
import random
random.shuffle(trainpair_idx)

train_pair = trainpair_idx[:int(len(trainpair_idx)*0.9)]
test_pair = trainpair_idx[int(len(trainpair_idx)*0.9):]
train_q1 = [p[0] for p in train_pair]
train_q2 = [p[1] for p in train_pair]
train_y = [p[2] for p in train_pair]
val_q1 = [p[0] for p in test_pair]
val_q2 = [p[1] for p in test_pair]
val_y = [p[2] for p in test_pair]

In [ ]:
model.fit([train_q1, train_q2], train_y, validation_data=([val_q1, val_q2], val_y), batch_size=128)

In [ ]:
filter_size = [1, 2, 3, 5]
layer_conv_c1 = [Conv1D(64, size, activation='relu', padding='same') for size in filter_size]

